# Load vector DB using normal Chroma DB code

In [17]:
import chromadb
import os
from chromadb.utils import embedding_functions
from dotenv import load_dotenv

In [18]:
load_dotenv() 
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
chroma_client = chromadb.PersistentClient(path="chromadb")
embeddings_fn = embedding_functions.OpenAIEmbeddingFunction(
    api_key=OPENAI_API_KEY
)

# Load from disk 
collection = chroma_client.get_collection(
    name="udaplay_games_collection",
    embedding_function=embeddings_fn
)

In [19]:
results = collection.query(
    query_texts=["halo"],
    n_results=2,
    include=["documents", "metadatas", "distances"]
)

print(results)

{'ids': [['015', '014']], 'embeddings': None, 'documents': [["[Xbox Series X|S] Halo Infinite (2021) - The latest installment in the Halo franchise, featuring Master Chief's return in a new open-world setting.", '[Xbox One] Minecraft (2014) - A sandbox game that allows players to build and explore infinite worlds, fostering creativity and adventure.']], 'uris': None, 'included': ['documents', 'metadatas', 'distances'], 'data': None, 'metadatas': [[{'Platform': 'Xbox Series X|S', 'Publisher': 'Xbox Game Studios', 'YearOfRelease': 2021, 'Name': 'Halo Infinite', 'Description': "The latest installment in the Halo franchise, featuring Master Chief's return in a new open-world setting.", 'Genre': 'First-person shooter'}, {'Platform': 'Xbox One', 'Genre': 'Sandbox, Survival', 'YearOfRelease': 2014, 'Name': 'Minecraft', 'Description': 'A sandbox game that allows players to build and explore infinite worlds, fostering creativity and adventure.', 'Publisher': 'Mojang Studios'}]], 'distances': [[

# Load vector DB using VectorStoreManager

In [20]:
from lib.agents import Agent
from lib.llm import LLM
from lib.state_machine import Run
from lib.messages import BaseMessage
from lib.tooling import tool
from lib.vector_db import VectorStoreManager
from lib.rag import RAG

In [21]:
db = VectorStoreManager(OPENAI_API_KEY, persistent_client=True, dir_path="chromadb")
db

VectorStoreManager():<chromadb.api.client.Client object at 0x0000021F6FFFEB40>

In [22]:
vector_store = db.get_or_create_store("udaplay_games_collection")
vector_store

In [23]:
type(vector_store)

lib.vector_db.VectorStore

In [26]:
vector_store.query("fighting game")["metadatas"]

[[{'Genre': 'Fighting',
   'Description': 'A crossover fighting game featuring characters from various Nintendo franchises battling it out in dynamic arenas.',
   'Publisher': 'Nintendo',
   'Platform': 'GameCube',
   'YearOfRelease': 2001,
   'Name': 'Super Smash Bros. Melee'},
  {'Publisher': 'Sony Interactive Entertainment',
   'Genre': 'Action-adventure',
   'Name': "Marvel's Spider-Man",
   'Platform': 'PlayStation 4',
   'YearOfRelease': 2018,
   'Description': 'An open-world superhero game that lets players swing through New York City as Spider-Man, battling iconic villains.'},
  {'YearOfRelease': 2010,
   'Description': 'A comprehensive racing simulator featuring a vast selection of vehicles and tracks, with realistic driving physics.',
   'Name': 'Gran Turismo 5',
   'Genre': 'Racing',
   'Platform': 'PlayStation 3',
   'Publisher': 'Sony Computer Entertainment'}]]

## Use predefined RAG class to load our vector store in 'chromadb'

In [8]:
rag_llm = LLM(
    model="gpt-4o-mini",
    temperature=0.3,
)

db = VectorStoreManager(OPENAI_API_KEY, persistent_client=True, dir_path="chromadb")
games_vector_store = db.get_or_create_store("udaplay_games_collection")
games_db_rag = RAG(
    llm=rag_llm,
    vector_store = games_vector_store
)

In [13]:
# Ask video game qns 
result:Run = games_db_rag.invoke("Who is in the game Halo Infinite?")
print(result.get_final_state()["answer"])

[StateMachine] Starting: __entry__
[StateMachine] Executing step: retrieve
[StateMachine] Executing step: augment
[StateMachine] Executing step: generate
[StateMachine] Terminating: __termination__
The game Halo Infinite features Master Chief as the main character.


In [14]:
result:Run = games_db_rag.invoke("What are some fighting games released in the year 2001?")
print(result.get_final_state()["answer"])

[StateMachine] Starting: __entry__
[StateMachine] Executing step: retrieve
[StateMachine] Executing step: augment
[StateMachine] Executing step: generate
[StateMachine] Terminating: __termination__
Some fighting games released in the year 2001 include Super Smash Bros. Melee.


In [15]:
result:Run = games_db_rag.invoke("What is the earliest and latest Gran Turismo games?")
print(result.get_final_state()["answer"])

[StateMachine] Starting: __entry__
[StateMachine] Executing step: retrieve
[StateMachine] Executing step: augment
[StateMachine] Executing step: generate
[StateMachine] Terminating: __termination__
The earliest Gran Turismo game is Gran Turismo (1997), and the latest Gran Turismo game mentioned is Gran Turismo 5 (2010).


In [16]:
result:Run = games_db_rag.invoke("Do a comparison of the game Halo Infinite and Super Mario 64")
print(result.get_final_state()["answer"])

[StateMachine] Starting: __entry__
[StateMachine] Executing step: retrieve
[StateMachine] Executing step: augment
[StateMachine] Executing step: generate
[StateMachine] Terminating: __termination__
Halo Infinite and Super Mario 64 are two iconic video games from different genres and eras, each with its own unique contributions to gaming.

**Super Mario 64 (1996)**:
- Genre: 3D platformer
- Gameplay: Players control Mario as he navigates through various worlds to rescue Princess Peach. The game is known for its innovative use of 3D graphics and open-world exploration, allowing players to choose how to approach levels.
- Impact: It set new standards for 3D platforming games and is often credited with popularizing the genre, influencing countless games that followed.

**Halo Infinite (2021)**:
- Genre: First-person shooter (FPS)
- Gameplay: Players take on the role of Master Chief in an open-world setting, featuring combat, exploration, and a rich narrative. The game emphasizes both singl